# Tutorial 3.2: Solving Mixed Discrete-Continuous Planning with Optimistic Search

In [1]:
from typing import List

import torch
import jacinle
import concepts.pdsketch as pds
from concepts.dsl.tensor_value import v

In [2]:
domain_string = r"""
(define (domain shapesetting)
(:types
  tile - object
  container - object
  pose - vector[float32, 2]
  cshape - vector[float32, 4]
  identifier - int64
)
(:predicates
  ;; part 1: boolean predicates
  (placed ?t - tile)
  ;; part 2: continuous-valued predicates
  (pose ?t - tile -> pose)
  (tile-identifier ?t - tile -> identifier)
  (container-shape ?c - container -> cshape)
  ;; functions
  (in-container ?ti - identifier ?tp - pose ?cshape - cshape)
  (left-of  ?t1 - identifier ?t2 - identifier ?tp1 - pose ?tp2 - pose)
  (right-of ?t1 - identifier ?t2 - identifier ?tp1 - pose ?tp2 - pose)
  (collision-free ?t1 - identifier ?t2 - identifier ?tp1 - pose ?tp2 - pose)
)
(:derived (left-of-t ?t1 - tile ?t2 - tile)
  (left-of (tile-identifier ?t1) (tile-identifier ?t2) (pose ?t1) (pose ?t2))
)
(:derived (right-of-t ?t1 - tile ?t2 - tile)
  (right-of (tile-identifier ?t1) (tile-identifier ?t2) (pose ?t1) (pose ?t2))
)
(:derived (collision-free-t ?t1 - tile ?t2 - tile)
  (collision-free (tile-identifier ?t1) (tile-identifier ?t2) (pose ?t1) (pose ?t2))
)
(:derived (in-container-t ?t - tile ?c - container)
  (in-container (tile-identifier ?t) (pose ?t) (container-shape ?c))
)
(:action place
  :parameters (?t - tile ?p - pose)
  :precondition (and
    (not (placed ?t))
  )
  :effect (and
    (placed ?t)
    (pose::assign ?t ?p)  ;; equivalent to (assign (pose ?t) ?p)
  )
)
; (:generator gen-in-container-pose
;  :parameters (?t - tile ?c - container)
;  :certifies (in-container-t ?t ?c)
;  :context (and
;    (tile-identifier ?t)
;    (container-shape ?c)
;  )
;  :generates (and
;    (pose ?t)
;  )
; )
;; the upper generate definition is equivalent to:
(:generator gen-in-container-pose
 :parameters (?ti - identifier ?tp - pose ?c - cshape)
 :certifies (in-container ?ti ?tp ?c)
 :context (and ?ti ?c)
 :generates (and ?tp)
)
)  ;; end of domain definition
"""

In [3]:
domain = pds.load_domain_string(domain_string)
domain.print_summary()

Domain shapesetting
  Types: dict{
    container: container
    cshape: cshape
    identifier: identifier
    pose: pose
    tile: tile
  }
  Functions: dict{
    collision-free: collision-free(?t1: identifier, ?t2: identifier, ?tp1: pose, ?tp2: pose) -> bool
    collision-free-t: collision-free-t[cacheable](?t1: tile, ?t2: tile) -> bool {
      collision-free(tile-identifier(V::?t1), tile-identifier(V::?t2), pose(V::?t1), pose(V::?t2))
    }
    container-shape: container-shape[observation, state, cacheable, static](?c: container) -> cshape
    in-container: in-container(?ti: identifier, ?tp: pose, ?cshape: cshape) -> bool
    in-container-t: in-container-t[cacheable](?t: tile, ?c: container) -> bool {
      in-container(tile-identifier(V::?t), pose(V::?t), container-shape(V::?c))
    }
    left-of: left-of(?t1: identifier, ?t2: identifier, ?tp1: pose, ?tp2: pose) -> bool
    left-of-t: left-of-t[cacheable](?t1: tile, ?t2: tile) -> bool {
      left-of(tile-identifier(V::?t1), tile-id

In [4]:
@pds.config_function_implementation(include_executor_args=True)
def left_of(executor, t1, t2, p1, p2):
    env: MyEnvironment = executor.environment

    assert env.tile_shapes[t1.item()] == 'box'
    assert env.tile_shapes[t2.item()] == 'box'

    # print('calling left_of', t1, t2, p1, p2)
    return p1[0] < p2[0]


def right_of(t1, t2, p1, p2):
    return p1[0] > p2[0]


def collision_free(t1, t2, p1, p2):
    return torch.ones_like(t1)


@pds.config_function_implementation(include_executor_args=True)
def in_container(executor, ti, tp, cshape):
    env: MyEnvironment = executor.environment
    assert env.tile_shapes[ti.item()] == 'box'

    x1, y1, w, h = cshape[..., 0], cshape[..., 1], cshape[..., 2], cshape[..., 3]
    x2, y2 = x1 + w, y1 + h

    x, y = tp[..., 0], tp[..., 1]
    return (x1 <= x) & (x <= x2) & (y1 <= y) & (y <= y2)


# if you don't want to use the "unwrap_values" helper function
# you can also use the following code:
# because all arguments are wrapped in a "Value" object.
# def left_of(t1, t2, p1, p2):
#     return p1.tensor[0] < p2.tensor[0]


def gen_in_container_pose(ti, cshape):
    return torch.rand(2) * cshape[..., 2:] + cshape[..., :2]


class MyEnvironment(object):
    def __init__(self, tile_shapes: List[str]):
        self.tile_shapes = tile_shapes

In [5]:
executor = pds.PDSketchExecutor(domain)
executor.register_function_implementation('predicate::left-of', left_of)
executor.register_function_implementation('predicate::right-of', right_of)
executor.register_function_implementation('predicate::collision-free', collision_free)
executor.register_function_implementation('predicate::in-container', in_container)
executor.register_function_implementation('generator::gen-in-container-pose', gen_in_container_pose)

env = MyEnvironment(['box', 'box'])
executor.environment = env

state, ctx = executor.new_state({
    'container': domain.types['container'], 
    'tile1': domain.types['tile'], 
    'tile2': domain.types['tile']
}, create_context=True)

# for all boolean predicates, write "true-valued" propositions as a list:
# ctx.define_predicates([ctx.placed('tile1')])
ctx.define_predicates([])

# for all continuous-valued predicates, use the "define_feature" function:
ctx.define_feature('pose', torch.zeros((2, 2), dtype=torch.float32))
ctx.define_feature('tile-identifier', torch.tensor([0, 1], dtype=torch.int64))
ctx.define_feature('container-shape', torch.tensor([
    [0, 0, 10, 10]
], dtype=torch.float32))  # tensor shape is [1, 4]

print('Initial state:', state)

goal = r'''(and
    (placed tile1)
    (placed tile2)
    (left-of-t tile1 tile2)
    (collision-free-t tile1 tile2)
    (in-container-t tile1 container)
    (in-container-t tile2 container)
)'''

print('Goal:', repr(executor.parse(goal)))

Initial state: State{
  states:
    - container-shape: Value[cshape, axes=[?c], tdtype=torch.float32, tdshape=(1, 4)]{
      tensor([[ 0.,  0., 10., 10.]])
    }
    - tile-identifier: Value[identifier, axes=[?t], tdtype=torch.int64, tdshape=(2,), quantized]{tensor([0, 1])}
    - pose: Value[pose, axes=[?t], tdtype=torch.float32, tdshape=(2, 2)]{
      tensor([[0., 0.],
              [0., 0.]])
    }
    - placed: Value[bool, axes=[?t], tdtype=torch.int64, tdshape=(2,), quantized]{tensor([0, 0])}
  objects: container - container, tile1 - tile, tile2 - tile
}
Goal: AndExpression<and(placed(OBJ::tile1), placed(OBJ::tile2), left-of-t(OBJ::tile1, OBJ::tile2), collision-free-t(OBJ::tile1, OBJ::tile2), in-container-t(OBJ::tile1, OBJ::container), in-container-t(OBJ::tile2, OBJ::container))>


In [6]:
action_list = [
    # v stands for "vector"
    domain.operators['place']('tile1', v(1, 1, dtype=domain.types['pose'])),
    domain.operators['place']('tile2', v(2, 2, dtype=domain.types['pose'])),
]

s = state.clone()
for action in action_list:
    succ, ns = executor.apply(action, s)
    assert succ
    s = ns

print('Landing state:', s)

print('Evaluating goal:', executor.execute(goal, state))

Landing state: State{
  states:
    - container-shape: Value[cshape, axes=[?c], tdtype=torch.float32, tdshape=(1, 4)]{
      tensor([[ 0.,  0., 10., 10.]])
    }
    - tile-identifier: Value[identifier, axes=[?t], tdtype=torch.int64, tdshape=(2,), quantized]{tensor([0, 1])}
    - pose: Value[pose, axes=[?t], tdtype=torch.float32, tdshape=(2, 2)]{
      tensor([[1., 1.],
              [2., 2.]])
    }
    - placed: Value[bool, axes=[?t], tdtype=torch.int64, tdshape=(2,), quantized]{tensor([1, 1])}
  objects: container - container, tile1 - tile, tile2 - tile
}
Evaluating goal: Value[bool, axes=[], tdtype=torch.int64, tdshape=(), quantized]{tensor(0)}


In [7]:
from concepts.pdsketch.planners.optimistic_search import construct_csp_from_optimistic_plan, ground_actions
from concepts.pdsketch.csp_solvers.dpll_sampling import csp_dpll_sampling_solve
from concepts.dsl.constraint import print_assignment_dict

action_list = [
    domain.operators['place']('tile1', '??'),
    domain.operators['place']('tile2', '??'),
]

optimistic_action_list, csp = construct_csp_from_optimistic_plan(executor, state, goal, action_list)

print('Optimistic action list:')
for action in optimistic_action_list:
    print(jacinle.indent_text(action))
print('-' * 40)

print('CSP:', csp)
print('-' * 40)

solution = csp_dpll_sampling_solve(executor, csp)
print('Solution:')
print_assignment_dict(solution)
print('-' * 40)

print('Insert back into the action list:')
solution_action_list = ground_actions(executor, optimistic_action_list, solution)
for action in solution_action_list:
    print(jacinle.indent_text(action))

optimistic left-of(tile-identifier(V::?t1), tile-identifier(V::?t2), pose(V::?t1), pose(V::?t2)) ()
optimistic collision-free(tile-identifier(V::?t1), tile-identifier(V::?t2), pose(V::?t1), pose(V::?t2)) ()
optimistic in-container(tile-identifier(V::?t), pose(V::?t), container-shape(V::?c)) ()
optimistic in-container(tile-identifier(V::?t), pose(V::?t), container-shape(V::?c)) ()
> /Users/jiayuanm/Projects/Concepts/concepts/pdsketch/executor.py(1000)visit_bool_expression()
    999 
-> 1000                     if determined is None:
   1001                         new_identifier = self.csp.new_var(BOOL)

ipdb> argv
OptValue[bool]{@5}
ipdb> c
Optimistic action list:
  action::place(?t=tile1, ?p=@0)
  action::place(?t=tile2, ?p=@1)
----------------------------------------
CSP: ConstraintSatisfactionProblem{
  Actionable Variables:
    @0 - pose
    @1 - pose
  Constraints:
    left-of(0, 1, [0.0, 0.0], [0.0, 0.0]) == O[bool]{@2}  # left-of(tile-identifier(V::?t1), tile-identifier(V::?t2),

AssertionError: 

In [ ]:
from concepts.pdsketch.planners.optimistic_search import optimistic_search

solution = optimistic_search(executor, state, goal, max_depth=3, verbose=True)
print('Solution:')
for action in solution:
    print(jacinle.indent_text(action))

In [ ]:
from concepts.pdsketch.planners.optimistic_search import optimistic_search_strips

# there is another "fancier" function called
# pds.optimistic_search_strips
# which does heurisitic computation (hFF-like) to prune the search tree.

solution = optimistic_search_strips(executor, state, goal, max_depth=3, verbose=True)
print('Solution:')
for action in solution:
    print(jacinle.indent_text(action))